# SS 2021 SEMINAR 09 Reinforcement Learning in der Sprachtechnologie
## QA: Examples in Python

### Announcements

#### Papers


#### Homework

* Update: Only 3 Homeworks needed for this seminar
        
* NEW DEADLINE: JUNE 23th

#### Today

* taking a closer look at Q-Learning on two examples: 1. Discrete Environment (TAX), 2. Continous Environment (CartPole) and how to tackle those

* QA

***

### A) Paper Presentation I: Judith

***
## Title: Ranking Sentences for Extractive Summarization with Reinforcement Learning 
(Shashi Narayan, Shay B. Cohen, Mirella Lapata, 2018)

### Link: 
[Ranking Sentences for Extractive Summarization with Reinforcement Learning](https://www.semanticscholar.org/paper/Ranking-Sentences-for-Extractive-Summarization-with-Narayan-Cohen/59562be2cf8e01e8b7bb7560cef56158ea171227)


### Summary:

The authors conceptualize <ins>extractive summarization as a sentence ranking task and propose a new training algorithm which globally optimizes the ROUGE evaluation metric through a reinforcement learning objective.</ins> They use their algorithm to train a neural summarization model on the CNN and DailyMail datasets and demonstrate experimentally that it outperforms extractive and abstractive systems when evaluated automatically and by humans.


***
***
### Problem/Task/Question:

#### <font color='blue'>TASK</font>: Single document summarization
+ =  producing a shorter version of a document while preserving its information content
+  (abstractive summarization: involves various text rewriting operations (e.g., substitution, deletion, reordering))
+ **extractive summarization**: identifying (and subsequently concatenating) the most important sentences in a document.



#### <font color='blue'>MOTIVATION</font>: previous models for extractive summarization
+ previous models (often) conceptualize <ins>extractive summarization as a sequence labeling task</ins> in which each label speciﬁes whether each document sentence should be included in the summary.
+ These models rely on recurrent neural networks to derive a meaning representation of the document which is then used to label each sentence, taking the previously labeled sentences into account. 
+ They are <ins>typically trained using cross-entropy loss in order to maximize the likelihood of the ground-truth labels</ins> 
+ **the authors criticize that**: 
 + these models do not learn to rank sentences based on their importance due to the absence of a ranking-based objective
 + there is a mismatch between the learning objective and the evaluation criterion, namely ROUGE, which takes the entire summary into account
 + models with cross-entropy training tend to generate wordy summaries with unnecessarily long sentences and redundant information
 
***
***

### Solution/Idea/Model/Approach:

#### <font color='blue'>SOLUTION</font>: Sentence Ranking with Reinforcement Learning
 + the authors adapt reinforcement learning to their formulation of extractive summarization to rank sentences for the summary generation: they <ins>globally optimize the ROUGE evaluation metric and learn to rank sentences for summary generation through a reinforcement learning objective</ins>
 + <ins> they train a model to predict the individual ROUGE score for each sentence in the document and then select the top m sentences with highest scores.</ins> <font color='grey'>(instead of maximizing the likelihood of the ground-truth labels with cross-entropy loss (like the previous models did))</font>
 +  they claim that this framework would make extractive models better at discriminating among sentences for the summarization task 



***
#### <font color='blue'>MODEL</font>: REFRESH  (REinFoRcement Learning-based Extractive Summarization)


#### -- Summarization as sentence ranking: 
+ <ins> a sentence is ranked high for selection if it often occurs in high scoring summaries</ins>
+ In detail: Given a document $D$ consisting of a sequence of sentences $(s_1,s_2 ,...,s_n )$, an extractive summarizer aims to produce a summary $S$ by selecting $m$ sentences from $D$ (where $m < n$).
+ For each sentence $s_i ∈ D$:
  + a  label $y_i  ∈ \{0,1\}$ is predicted (where 1 means that $s_i$ should be included in the summary) 
  + and a score $p(y_i|s_i,D , θ)$ is assigned, quantifying $s_i$’s relevance to the summary <font color='grey'>(Model parameters are denoted by $θ$)</font>
+ They estimate $p(y_i|s_i,D , θ)$ using a neural network model and assemble a summary $S$ by selecting $m$ sentences with top $p(1|s_i,D , θ)$ scores.
 
 
#### -- Components of the model:

![alt text](https://raw.githubusercontent.com/clause-bielefeld/SS_2021_SEMINAR_Reinforcement_Learning_in_der_Sprachtechnologie/main/materials/images/figure_1.png "")

+ **sentence encoder** (the core component): a convolutional neural network (CNN) to encode sentences into continuous representations --> returns sentence embeddings 
 +  *<font color='grey'>they use temporal narrow convolution by applying a kernel ﬁlter $K$ of width $h$ to a window of $h$ words in sentence $s$ to produce a new feature. This ﬁlter is applied to each possible window of words in $s$ to produce a feature map $f ∈ R^{k−h+1}$ where $k$ is the sentence length. They then apply max-pooling over time over the feature map $f$ and take the maximum value as the feature corresponding to this particular ﬁlter $K$. They  use multiple kernels of various sizes and each kernel multiple times to construct the representation of a sentence.* 
 + *<font color='grey'>In the figure, kernels of size 2 (red) and 4 (blue) are applied three times each. Max-pooling over time yields two feature lists $f^{K_2}$ and $f^{K_4}∈ R^3$ . The ﬁnal sentence embeddings have six dimensions.*


+ **document encoder**: a recurrent neural network (RNN) to compose a sequence of sentences to obtain a document representation
 + *<font color='grey'>they feed sentences in reverse order to make sure that the network also considers the top sentences of the document which are particularly important for summarization*
 + *<font color='grey'> is implemented with LSTM (Long Short-Term Memory) to avoid the vanishing gradient problem when training long sequences*
 

+ **sentence extractor**: another RNN to sequentially label each sentence in a document with 1 (relevant for the summary) or 0 (otherwise) 
 + *<font color='grey'>It is implemented with LSTM cells and a softmax layer*  
 + *<font color='grey'>reads the sentence and makes a binary prediction, conditioned on the document representation (obtained from the document encoder) and the previously labeled sentences. This way, the sentence extractor is able to identify locally and globally important sentences within the document.*
 + *<font color='grey'>they rank the sentences in a document $D$ by $p(y_i = 1 | s_i ,D,θ)$, which is the conﬁdence scores assigned by the softmax layer of the sentence extractor.*




***

#### -- they cast this summarization model in the <font color='blue'>REINFORCEMENT LEARNING PARADIGM</font>: 
 + the **model** can be viewed as the <font color='green'>**agent**</font> which interacts with an <font color='green'>**environment**</font> consisting of **documents** 
 + the <font color='green'>**actions**</font> are **relevance scores** which lead to sentence ranking
 
 
 + the training algorithm performs sentence ranking using **ROUGE** as the <font color='green'>**reward function**</font>.
   + As the reward function they use mean $F_1$ of ROUGE-1, ROUGE-2, and ROUGE-L.
   + *<font color='grey'> ROUGE-1 and ROUGE-2 measure unigram and bigram overlap between model output and reference (gold standard summary) and are meant to assess informativeness*
   + *<font color='grey'> ROUGE-L measures the longest common subsequence between model output and referenceis and is meant to assess ﬂuency*
 + the algorithm explores the space of candidate summaries while learning to optimize the reward function
 + During training, the model combines the maximum-likelihood cross-entropy loss with rewards from policy gradient reinforcement learning to directly optimize the evaluation metric relevant for the summarization task
  


 + **<font color='green'>Policy Learning**</font>:
 + **<font color='green'>1.</font> At ﬁrst, the agent is initialized randomly, it reads document $D$ and predicts a relevance score for each sentence $s_i ∈ D$ using policy $p(y_i | s_i, D, θ)$**
 + **<font color='green'>2.</font>  Once the agent is done reading the document, a summary with labels $\hat{y}$ is sampled out of the ranked sentences.**
 + **<font color='green'>3.</font>  The agent is then given a reward $r$ according to how well the extract resembles the gold-standard summary (with the reward function).**
 
 
 + **They update the agent using the [REINFORCE algorithm (Williams, 1992)](https://people.cs.umass.edu/~barto/courses/cs687/williams92simple.pdf)** which aims to minimize the negative expected reward: $L(θ) = –E_{\hat{y}∼p_θ}[r(\hat{y})]$ <font color='grey'>($p_θ$ stands for $p(y |D, θ)$)</font>
   + <font color='grey'>REINFORCE is based on the observation that the expected gradient of a non-differentiable reward function (ROUGE, in this case) can be computed as follows: $∇L(θ) = –E_{\hat{y}∼p_θ}[r(\hat{y})∇log p(\hat{y}|D,θ]$
   + The authors approximate the expected gradient using a single sample $\hat{y}$ from $p_θ$ for each training example in a batch:
   + $∇L(θ) ≈ r(\hat{y})∇log p(\hat{y}|D,θ) 
   ≈ r(\hat{y}) \sum_{i=1}^{n}∇log p(\hat{y_i}|s_i,D,θ)$*
   + they limit the search space of $\hat{y}$ in this Equation to the set of largest probability samples $\hat{Y}$: They approximate $\hat{Y}$ by the k extracts which receive highest ROUGE scores</font>
 + the objective is to learn to discriminate among sentences with respect to how often they occur in high scoring summaries
 
 
 + **In other words**: 
 + they assemble candidate summaries by ﬁrst selecting $p$ sentences from the document which on their own have high ROUGE scores. They then generate all possible combinations of $p$ sentences subject to maximum length $m$ and evaluate them against the gold summary.
    + Summaries are ranked according to $F_1$ by taking the mean of ROUGE-1, ROUGE-2, and ROUGE-L. 
    + $\hat{Y}$ contains these top $k$ candidate summaries. During training, they sample $\hat{y}$  from $\hat{Y}$ instead of $p(\hat{y}|D, θ)$

     
   
+ As a result, **reinforcement learning helps extractive summarization in two ways**: 
  + **(a)** it directly optimizes the evaluation metric instead of maximizing the likelihood of the ground-truth label
  + **(b)** it makes their model better at discriminating among sentences: a sentence is ranked high for selection if it often occurs in high scoring summaries
    


***
#### <font color='blue'>EXPERIMENTAL SETUP</font>: 

#### -- Datasets:
+ evaluated their model on the <ins> CNN and DailyMail news highlights datasets</ins> (these have been used as testbeds for the evaluation of neural summarization systems) and did not anonymize entities or lower case tokens
+ assumed that the <ins>“story highlights” associated with each article are gold-standard abstractive summaries </ins>
  + during training they are used to generate high scoring extracts and to estimate rewards for them
  + during testing, they are used as reference summaries to evaluate the model


#### <font color='grey'>-- Some Implementation Details:
+ <font color='grey'> generated extracts by selecting three sentences (m = 3) for CNN articles and four sentences (m = 4) for DailyMail articles (because gold highlights in the CNN/DailyMail validation sets are 2.6/4.2 sentences long)
+ For both datasets, they estimated high-scoring extracts using 10 document sentences (p = 10) with highest ROUGE scores
+ They used the One Billion Word Benchmark corpus with the skip-gram model to train word embeddings.
+ For the sentence encoder, they used a list of kernels of widths 1 to 7, each with output channel size of 50. The sentence embedding size in their model was 350.
+ For the recurrent neural network component in the document encoder and sentence extractor, they used a single-layered LSTM network with size 600. 
+ they performed minibatch cross-entropy training with a batch size of 20 documents for 20 training epochs. After each epoch, they evaluated their model on the validation set and chose the best performing model for the test set. 
+ Their system is implemented in TensorFlow*

***
***
### Main Results/Findings: 

#### <font color='blue'>AUTOMATIC EVALUATION</font>: 

+ report F1 ROUGE to evaluate summarization quality 
+ report ROUGE-1 and ROUGE-2 (unigram and bigram overlap) to assess informativeness 
+ report ROUGE-L (longest common subsequence) to assess ﬂuency


+ compare their model REFRESH against:
  + a baseline which simply selects the ﬁrst leading sentences from each document (LEAD)
  + two extractive neural models which are both trained with cross-entropy loss:
    + Cheng and Lapata (2016) (who train on individual labels; i.e. one sentence, one label)
    + Nallapati et al. (2017) (who use collective labels; i.e. one label for a few sentences)
  + the abstractive systems of Chen et al. (2016), Nallapati et al. (2016), See et al. (2017), and Tan and Wan (2017).


#### -- Results:

![alt text](https://raw.githubusercontent.com/clause-bielefeld/SS_2021_SEMINAR_Reinforcement_Learning_in_der_Sprachtechnologie/main/materials/images/table_2.png "")


+ The results in this Table2 show that REFRESH is superior to the LEAD baseline and extractive systems across datasets and metrics. 
+ their model „outperforms“ both extractive and abstractive systems
+ results show that cross-entropy training is not well-suited to the summarization task
+ state-of-the-art abstractive systems lag behind extractive ones when the latter are globally trained



#### <font color='blue'>TWO HUMAN EVALUATIONS</font>: 



#### -- Experiment 1: to assess which type of summary participants prefer

+ they <ins>compared extractive and abstractive systems: Participants read the articles and were asked to rank the summaries from best (1) to worst (4) in order of informativeness and ﬂuency.</ins>
+ participants were presented with a news article and summaries generated by these systems: the LEAD baseline, abstracts from See et al. (2017) (abstractive) and extracts from REFRESH and also. The authors also included the human-authored highlights (GOLD). 
+ The authors randomly selected 10 articles from the CNN test set and 10 from the DailyMail test set. The study was completed by ﬁve participants, all native or proﬁcient English speakers. Each participant was presented with the 20 articles. The order of summaries to rank was randomized per article and the order of articles per participant.

#### -- Experiment 2: to assess how much key information from the document is preserved in the summary

+ <ins> followed a question-answering (QA) paradigm </ins> and created a set of questions based on the gold summary
+ They <ins> examined whether participants were able to answer these questions by reading system summaries alone</ins> (the more questions a system can answer, the better it is at summarizing the document as a whole
+ They used the same 20 documents  as in the ﬁrst elicitation study. The authors wrote multiple factbased question-answer pairs for each gold summary without looking at the document.
+ Five participants were shown summaries from three systems: the LEAD baseline, the abstractive system of See et al. (2017), and REFRESH. 
+ scoring mechanism: a correct answer was marked with a score of one, partially correct answers with a score of 0.5, and zero otherwise. The ﬁnal score for a system is the average of all its question scores. 

![alt text](https://raw.githubusercontent.com/clause-bielefeld/SS_2021_SEMINAR_Reinforcement_Learning_in_der_Sprachtechnologie/main/materials/images/figure_2.png "")



#### -- Results:

![alt text](https://raw.githubusercontent.com/clause-bielefeld/SS_2021_SEMINAR_Reinforcement_Learning_in_der_Sprachtechnologie/main/materials/images/table_3.png "")

+ **experiment 1**: REFRESH is ranked second best (after GOLD)
+ **experiment 2**: REFRESH is ranked best (based on summaries generated by REFRESH, participants can answer 66.34% of questions correctly).
+ this „overwhelmingly“ shows that human subjects ﬁnd the summaries produced bei REFRESH more informative and complete.


***
***
### Critical Discussion: 

+ <font color='green'>**+**</font> the article is written understandable and with a clear structure
+ <font color='green'>**+**</font> In my opinion the authors have explained in an comprehensible way why they handle extractive summarization differently from previous models 
+ <font color='green'>**+**</font> I liked that they examined both automatic and human evaluation
+ <font color='green'>**+**</font>  ... 


+ <font color='red'>**-**</font> only one small example for a summary produced by REFRESH was given 
+ <font color='red'>**-**</font> only five participants were chosen for evaluation and we don’t get a lot of information about the participants (such as age, gender, relationship to the authors etc.)
+ <font color='red'>**-**</font> the authors authors wrote the questions for experiment 2 themselves (even if they did it "without looking at the document", I still find this questionable)
+ <font color='red'>**-**</font>  ... 

***

### Discussion



***

## C) Practice

 **RL in Python**

In this seminar, we will take a closer look at two examples. 

The first example deals with the TAXI environment, which is an environment with a **discrete state space**.

The second example deals with the CARTPOLE environment, which is an environment with a **continous state space**. 

### Taxi-V3 Environment

Find the environment here: https://gym.openai.com/envs/Taxi-v3/

For solving the Taxi environment, Q-Learning with a discrete Q-Table can be applied. 

In the following there are 2 examples shown. 

Example 1 is a simple, straight forward implementation of the learning algorithm NOT FOLLOWING the openai gym interfaces. 

Example 2 is an implementation following the interfaces given by openai gym, which are widely accepted within the commumity. The advantage of following a programming style due to those interfaces is, that you can test and run your agents on different environments literally in a plug and play mode. 

**MY TIPP FOR YOU:** If you are new to reinforcement learning and object oriented programming at all, try to implement the simpler version first and then try to understand the more abstract version. In you homework it is absolutely okay if you solve an environment with a simple, straight forward implementation of the Q-Learning Algorithm. 

### Straight Forward Approach

In [3]:
import numpy as np
import gym 
import random 
import time
from abc import ABC, abstractmethod
from IPython.display import clear_output

In [4]:
# get an instance of the frozen lake gym environment
env = gym.make("Taxi-v3")

In [6]:
# get some information from the environment
action_space = env.action_space
print(action_space)
action_space_size = env.action_space.n
print(action_space_size)
state_space = env.observation_space 
print(state_space)
state_space_size = env.observation_space.n
print(state_space_size)

Discrete(6)
6
Discrete(500)
500


In [7]:
# Q-TABLE
# build our action-value table | Q-TABLE
# as you already know, the q-table looks like this
# state | action_space

q_table = np.zeros((state_space_size, action_space_size))
#print(q_table)

[[0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 ...
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]]


In [8]:
# Training Parameters
num_episodes = 10000
max_steps_per_episode = 100
num_test_episodes = 3     

# q-learning | update parameters
learning_rate = 0.1
discount_rate = 0.99

# exploration-exploitation trade off
exploration_rate = 1
max_exploration_rate = 1
min_exploration_rate = 0.01
exploration_decay_rate = 0.001

In [9]:
# Q-LEARNING

# collect rewards somewhere to visualize our learning curve
rewards_of_all_episodes = []

# Training Loop
for episode in range(num_episodes):
    # reset/initialize the environment first
    state = env.reset()
    # set done back to false at the beginning of an episode
    done = False
    # reset our rewards collector | return for the beginning episode
    rewards_current_episode = 0
    
    for step in range(max_steps_per_episode):
        # select an action
        # use our exploration exploitation trade off -> do we explore or exploit in this timestep ?
        exploration_rate_threshold = random.uniform(0,1)
        if(exploration_rate_threshold > exploration_rate):
            action = np.argmax(q_table[state, : ])
        else:
            action = env.action_space.sample()
            
        new_state, reward, done, info = env.step(action)
        
        # Update Q-Table Q(s,a) using the bellman update  
        q_table[state, action] = q_table[state, action] * (1 - learning_rate) + learning_rate * (reward + discount_rate * np.max(q_table[new_state, : ]))
        
        # update the state to the new state
        state = new_state
        # collect the reward
        rewards_current_episode += reward
        
        if (done == True):
            break
    
    # after we finish an episode, make sure to update the exploration rate
    # decay the exploration rate the longer the time goes on
    exploration_rate = min_exploration_rate + (max_exploration_rate - min_exploration_rate) * np.exp(-exploration_decay_rate*episode)
    # append our rewards for this episode for learning curve
    rewards_of_all_episodes.append(rewards_current_episode)

In [10]:
# LEARNING STATISTICS 
# for each episode print the stats of the episode
rewards_per_thousand_episodes = np.split(np.array(rewards_of_all_episodes),num_episodes/1000)
count = 1000
print('*****INFO: average reward per thousand episodes: ***** \n')
for reward in rewards_per_thousand_episodes:
    print(count, ": ", str(sum(reward/1000)))
    count += 1000
    
# print our learned q-table
print("\n\n ***** Q-TABLE ***** \n")
print(q_table)

*****INFO: average reward per thousand episodes: ***** 

1000 :  -251.62699999999995
2000 :  -40.56400000000015
3000 :  2.023999999999995
4000 :  5.467999999999978
5000 :  6.982999999999963
6000 :  7.306999999999973
7000 :  7.374999999999958
8000 :  7.208999999999966
9000 :  7.34199999999997
10000 :  7.370999999999965


 ***** Q-TABLE ***** 

[[ 0.          0.          0.          0.          0.          0.        ]
 [-3.58354789  1.50738533 -2.67946757 -0.72508374  9.6220697  -7.85762471]
 [ 3.24488086  8.5430262   0.90119722  6.05827363 14.11880599 -3.88260399]
 ...
 [-1.09370698  4.21394742 -1.13587692 -1.42989285 -8.20037517 -7.85013748]
 [-3.08914083 -3.04936186 -3.26431454  5.38236448 -9.84217015 -9.37361561]
 [-0.23842085 -0.31662019  0.12178955 16.27783037 -2.68370758 -1.24277331]]


In [12]:
# EVALUATION | TESTING | watching our agent play
for episode in range(num_test_episodes):
    state = env.reset()
    done = False
    print("INFO:*****EPISODE ", episode+1, "\n\n\n")
    time.sleep(1)
    
    for step in range(max_steps_per_episode):
        clear_output(wait=True)
        env.render()
        time.sleep(0.3)
        
        action = np.argmax(q_table[state, :])
        new_state, reward, done, info = env.step(action)
        
        if done:
            clear_output(wait=True)
            env.render()
            if reward == 1: # check reward from environment for correct display
                print("INFO: ***** agent reached the goal. *****")
                time.sleep(3)
            else:
                print("INFO: ***** agent did not reach the goal.")
                time.sleep(3)
            clear_output(wait=True)
            break
        
        state = new_state

env.close()

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (Dropoff)
INFO: ***** agent did not reach the goal.


### OpenAI Interface Conform Approach

In [13]:
# DOMAIN = the data_object (JSON SERIALIZABLE)
class Environment():
    """The main Environment class. It encapsulates an environment with
    arbitrary behind-the-scenes dynamics. An environment can be
    partially or fully observed.
    The main API methods that users of this class need to know are:
        step
        reset
        render
        close
        seed
    And set the following attributes:
        action_space: The Space object corresponding to valid actions
        observation_space: The Space object corresponding to valid observations
        reward_range: A tuple corresponding to the min and max possible rewards
    Note: a default reward range set to [-inf,+inf] already exists. Set it if you want a narrower range.
    The methods are accessed publicly as "step", "reset", etc...
    """
    # Set this in SOME subclasses
    metadata = {'render.modes': []}
    reward_range = (-float('inf'), float('inf'))
    spec = None

    def __init__(self, action_space=None, observation_space=None):
        # Set variables
        self.action_space = action_space
        self.observation_space = observation_space
    
# REPOSITORY = the functionality interface
class EnvironmentRepository(ABC):
    @abstractmethod
    def step(self, action):
        """Run one timestep of the environment's dynamics. When end of
        episode is reached, you are responsible for calling `reset()`
        to reset this environment's state.
        Accepts an action and returns a tuple (observation, reward, done, info).
        Args:
            action (object): an action provided by the agent
        Returns:
            observation (object): agent's observation of the current environment
            reward (float) : amount of reward returned after previous action
            done (bool): whether the episode has ended, in which case further step() calls will return undefined results
            info (dict): contains auxiliary diagnostic information (helpful for debugging, and sometimes learning)
        """
        pass

    @abstractmethod
    def reset(self):
        """Resets the environment to an initial state and returns an initial
        observation.
        Note that this function should not reset the environment's random
        number generator(s); random variables in the environment's state should
        be sampled independently between multiple calls to `reset()`. In other
        words, each call of `reset()` should yield an environment suitable for
        a new episode, independent of previous episodes.
        Returns:
            observation (object): the initial observation.
        """
        pass

    @abstractmethod
    def render(self, mode='human'):
        """Renders the environment.
        The set of supported modes varies per environment. (And some
        environments do not support rendering at all.) By convention,
        if mode is:
        - human: render to the current display or terminal and
          return nothing. Usually for human consumption.
        - rgb_array: Return an numpy.ndarray with shape (x, y, 3),
          representing RGB values for an x-by-y pixel image, suitable
          for turning into a video.
        - ansi: Return a string (str) or StringIO.StringIO containing a
          terminal-style text representation. The text can include newlines
          and ANSI escape sequences (e.g. for colors).
        Note:
            Make sure that your class's metadata 'render.modes' key includes
              the list of supported modes. It's recommended to call super()
              in implementations to use the functionality of this method.
        Args:
            mode (str): the mode to render with
        Example:
        class MyEnv(Env):
            metadata = {'render.modes': ['human', 'rgb_array']}
            def render(self, mode='human'):
                if mode == 'rgb_array':
                    return np.array(...) # return RGB frame suitable for video
                elif mode == 'human':
                    ... # pop up a window and render
                else:
                    # just raise an exception
                    super(MyEnv, self).render(mode=mode)
        """
        pass

    @abstractmethod
    def close(self):
        """Override close in your subclass to perform any necessary cleanup.
        Environments will automatically close() themselves when
        garbage collected or when the program exits.
        """
        pass


In [14]:
# REPOSITORY IMPLEMENTATION = the way how you would like to implement it
class EnvironmentRepositoryImpl(EnvironmentRepository):
    # Initialize / Instance Attributes
    def __init__(self, environment):
        # Set variables
        self.data_object = environment
        print('Environment initialized')

    def step(self, action):
        state = self.data_object.step(action)
        return state

    def reset(self):
        state = self.data_object.reset()
        return state

    def render(self):
        self.data_object.render()

    def close(self):
        state = self.data_object.close()

    def get_action_space(self):
        # get action space from api of the playground or via js in browser using selenium
        action_space = self.data_object.action_space
        return action_space

    def get_observation_space(self):
        # get observation space of the playground from api or via js in browser using selenium
        observation_space = self.data_object.observation_space
        return observation_space

In [15]:
# DOMAIN
class Agent():
    # class variables
    agent_variable = ""
    # class methods
    def __init__(self, agent_variable=""):
        self.agent_variable = agent_variable
        
# REPOSITORY
class AgentRepository(ABC):
    @abstractmethod
    def get_action(self, state): # This is the agent's POLICY, if you will
        """ Agent gets a state as input and returns an action 
        """
        pass

In [16]:
# REPOSITORY IMPLEMENTATION
class AgentRepositoryImpl(AgentRepository):
    # Initializer / Instance Attributes
    def __init__(self, environment, agent):
        # Set variables
        self.data_object = agent
        self.environment = environment
        self.action_space = self.environment.action_space
        self.observation_space = self.environment.observation_space
        self.action_space_size = self.environment.action_space.n
        self.observation_space_size = self.environment.observation_space.n
        # INIT Q-TABLE
        self.q_table = np.zeros((self.observation_space_size, self.action_space_size))
        # INIT AGENT PARAMETERS
        self.learning_rate = 0.7           # Learning rate
        self.discount_rate = 0.618         # Discounting rate
        self.exploration_rate = 1.0        # Exploration rate
        self.max_exploration_rate = 1.0    # Exploration probability at start
        self.min_exploration_rate = 0.01   # Minimum exploration probability 
        self.exploration_decay_rate = 0.01 # Exponential decay rate for exploration probability
        print('Agent initialized.')

    def get_action(self, state):
        #EXPLORATION-EXPLOITATION TRADE OFF
        exploration_rate_threshold = random.uniform(0,1)
        if(exploration_rate_threshold > self.exploration_rate):
            # get action from q table
            action = np.argmax(self.q_table[state, : ])
        else:
            # get random action
            action = self.get_random_action()
        return action
    
    def get_random_action(self):
        #action_set = random.sample(self.action_space, 1)
        #action = action_set[0]
        action = self.action_space.sample()
        return action
    
    def update_q_table(self, state, action, reward, new_state):
        self.q_table[state, action] = self.q_table[state, action] * (1 - self.learning_rate) + self.learning_rate * (reward + self.discount_rate * np.max(self.q_table[new_state, : ]))
        
    def update_exploration_rate(self, episode_num):
        self.exploration_rate = self.min_exploration_rate + (self.max_exploration_rate - self.min_exploration_rate) * np.exp(-self.exploration_decay_rate*episode_num)
    
    def get_exploit_action(self, state):
        action = np.argmax(self.q_table[state, : ])
        return action

In [17]:
# Training Parameters
num_episodes = 50000        # Total episodes
max_steps_per_episode = 100 # Max steps per episode
num_test_episodes = 5     # Total test episodes

In [18]:
# Setting up the Environment

# get the environment
env = gym.make("Taxi-v3")

# get some information from the environment
action_space = env.action_space
print(action_space)
action_space_size = env.action_space.n
print(action_space_size)
observation_space = env.observation_space 
print(observation_space)
observation_space_size = env.observation_space.n
print(observation_space_size)
reward_range = env.reward_range
print(reward_range)

environment_data_object = env #Environment(action_space, observation_space)
environment = EnvironmentRepositoryImpl(environment_data_object)

# Setting up the Agent
agent_data_object = Agent()
agent = AgentRepositoryImpl(environment_data_object, agent_data_object)

Discrete(6)
6
Discrete(500)
500
(-inf, inf)
Environment initialized
Agent initialized.


In [19]:
# TRAINING
# collect rewards somewhere to visualize our learning curve
rewards_of_all_episodes = []

# Training Loop
for episode in range(num_episodes):
    # reset/initialize the environment first
    state = environment.reset()
    # set done back to false at the beginning of an episode
    done = False
    # reset our rewards collector | return for the beginning episode
    rewards_current_episode = 0
    
    for step in range(max_steps_per_episode):
        # select an action
        # use our exploration exploitation trade off -> do we explore or exploit in this timestep ?
        action = agent.get_action(state)
            
        new_state, reward, done, info = environment.step(action)
        
        # Update Q-Table Q(s,a) using the bellman update  
        agent.update_q_table(state, action, reward, new_state)
        
        # update the state to the new state
        state = new_state
        # collect the reward
        rewards_current_episode += reward
        
        if (done == True):
            break
    
    # after we finish an episode, make sure to update the exploration rate
    # decay the exploration rate the longer the time goes on
    agent.update_exploration_rate(episode)
    # append our rewards for this episode for learning curve
    rewards_of_all_episodes.append(rewards_current_episode)

In [20]:
# LEARNING STATISTICS 
# for each episode print the stats of the episode
rewards_per_thousand_episodes = np.split(np.array(rewards_of_all_episodes),num_episodes/1000)
count = 1000
print('*****INFO: average reward per thousand episodes: ***** \n')
for reward in rewards_per_thousand_episodes:
    print(count, ": ", str(sum(reward/1000)))
    count += 1000
    
# print our learned q-table
print("\n\n ***** Q-TABLE ***** \n")
print(agent.q_table)

*****INFO: average reward per thousand episodes: ***** 

1000 :  -45.841999999999764
2000 :  7.287999999999954
3000 :  7.259999999999967
4000 :  7.023999999999967
5000 :  7.145999999999961
6000 :  7.245999999999962
7000 :  7.203999999999955
8000 :  7.099999999999966
9000 :  7.40999999999997
10000 :  7.340999999999965
11000 :  7.515999999999975
12000 :  7.4629999999999646
13000 :  7.3719999999999715
14000 :  7.333999999999969
15000 :  7.510999999999968
16000 :  7.267999999999974
17000 :  7.360999999999959
18000 :  7.371999999999952
19000 :  7.421999999999969
20000 :  7.513999999999961
21000 :  7.554999999999962
22000 :  7.2889999999999615
23000 :  7.44399999999997
24000 :  7.575999999999966
25000 :  7.469999999999965
26000 :  7.227999999999968
27000 :  7.430999999999969
28000 :  7.321999999999962
29000 :  7.374999999999972
30000 :  7.409999999999963
31000 :  7.292999999999961
32000 :  7.396999999999964
33000 :  7.383999999999964
34000 :  7.428999999999962
35000 :  7.465999999999966
3600

In [21]:
# EVALUATION | TESTING | watching our agent play
for episode in range(num_test_episodes):
    state = environment.reset()
    done = False
    print("INFO:*****EPISODE ", episode+1, "\n\n\n")
    time.sleep(1)
    
    for step in range(max_steps_per_episode):
        clear_output(wait=True)
        environment.render()
        time.sleep(0.3)
        
        action = agent.get_exploit_action(state)
        new_state, reward, done, info = environment.step(action)
        
        if done:
            clear_output(wait=True)
            environment.render()
            if reward == 20:
                print("INFO: ***** agent reached the goal. *****")
                time.sleep(3)
            else:
                print("INFO: ***** agent missed the goal.")
                time.sleep(3)
            clear_output(wait=True)
            break
        
        state = new_state

environment.close()

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (Dropoff)
INFO: ***** agent reached the goal. *****


### CartPole Environment

Find the environment here: https://gym.openai.com/envs/CartPole-v0/

For solving the CartPole environment, Q-Learning with two different approaches can be applied. 

The first approach would be to **discretize** the state space of the environment and based on this build a Q-Table following the Q-Learning algorithm. 

The second approach would be to **approximate** the Q-table with a neural network, due to the fact that a continous state space would lead to an infinite q-table. 

In the following there are 2 examples shown. 

Example 1 is an implementation of the Q-Learning algorithm using a discretization of the state space of the cartpole environment. 

Example 2 is an implementation of the Q-Learning algorithm that approximates the (infinite) q-table with a neural network. 

**MY TIPP FOR YOU:** If you are new to reinforcement learning and object oriented programming try to discretize your environments and use a simple, straigh forward implementation of the Q-Learning environment. If you are an advanced programmer and familiar to concepts of RL and ML when dealing with a continous state space of your environment use a neural network for approximating the q-table and build it into an agent which fits the openai interfaces. 

#### Discretization Approach

In [2]:
import numpy as np
import gym 
import random 
import time
from abc import ABC, abstractmethod
from IPython.display import clear_output

In [3]:
# get an instance of the frozen lake gym environment
env = gym.make("CartPole-v0")

In [4]:
# get some information from the environment
action_space = env.action_space
print(action_space)
action_space_size = env.action_space.n
print(action_space_size)
state_space = env.observation_space 
print(state_space)
state_space_size = env.observation_space.n
print(state_space_size)

Discrete(2)
2
Box(-3.4028234663852886e+38, 3.4028234663852886e+38, (4,), float32)


AttributeError: 'Box' object has no attribute 'n'

In [ ]:
?env.env

In [15]:
# State Discretizer
def discretize_range(lower_bound, upper_bound, num_bins):
    return np.linspace(lower_bound, upper_bound, num_bins + 1)[1:-1]

# Discretize the continuous state space for each of the 4 features.
num_discretization_bins = 7
state_bins = [
    # Cart position.
    discretize_range(-4.8, 4.8, num_discretization_bins),
    # Cart velocity.
    discretize_range(-3.0, 3.0, num_discretization_bins),
    # Pole angle.
    discretize_range(-0.5, 0.5, num_discretization_bins),
    # Tip velocity.
    discretize_range(-2.0, 2.0, num_discretization_bins)
]
print(state_bins)
max_bins = max(len(bin) for bin in state_bins)
print(max_bins)
num_states = (max_bins + 1) ** len(state_bins)
print(num_states)
# our state space therefore has a size of 7x7x7x7 ~2500
state_space_size = num_states

# lets see how that looks like in practice
state = env.reset()
print(state)
print(state[0])

# discretize a given state into our state model, found here: https://www.statology.org/numpy-digitize/
def discretize_value(value, bins):
    return np.digitize(x=value, bins=bins)

def discretize_state(observation):
        # Discretize the observation features and reduce them to a single integer
        #for i, feature in enumerate(observation):
            #getting the bins
            #print(state_bins[i]) 
            # extending to the max number of bins
            #print(state_bins[i] * ((max_bins + 1)))
            # putting it into the power of the decimal place 
            #print(state_bins[i] * ((max_bins + 1)) ** i)
            # this encoding enables us to reach that: first states encodes one potency, second state encodes ten potency, third state encodes hundred potency, ...
            #print(discretize_value(feature, state_bins[i]) * ((max_bins + 1) ** i))
            
        state = sum(
            discretize_value(feature, state_bins[i]) * ((max_bins + 1) ** i)
            for i, feature in enumerate(observation)
        )
        return state
    
test = discretize_state(state)
print(test)

[array([-3.42857143, -2.05714286, -0.68571429,  0.68571429,  2.05714286,
        3.42857143]), array([-2.14285714, -1.28571429, -0.42857143,  0.42857143,  1.28571429,
        2.14285714]), array([-0.35714286, -0.21428571, -0.07142857,  0.07142857,  0.21428571,
        0.35714286]), array([-1.42857143, -0.85714286, -0.28571429,  0.28571429,  0.85714286,
        1.42857143])]
6
2401
[ 0.04595299  0.03132225 -0.03783019  0.04184446]
0.045952988930203595
1200


In [ ]:
# Q-TABLE
# build our action-value table | Q-TABLE
# as you already know, the q-table looks like this
# state | action_space

q_table = np.zeros((state_space_size, action_space_size))
#print(q_table)

In [ ]:
# Training Parameters
num_episodes = 10000
max_steps_per_episode = 100
num_test_episodes = 1     

# q-learning | update parameters
learning_rate = 0.1
discount_rate = 0.99

# exploration-exploitation trade off
exploration_rate = 1
max_exploration_rate = 1
min_exploration_rate = 0.01
exploration_decay_rate = 0.001

In [ ]:
# Q-LEARNING

# collect rewards somewhere to visualize our learning curve
rewards_of_all_episodes = []

# Training Loop
for episode in range(num_episodes):
    # reset/initialize the environment first
    state = env.reset()
    # discretize state
    state = discretize_state(state)
    # set done back to false at the beginning of an episode
    done = False
    # reset our rewards collector | return for the beginning episode
    rewards_current_episode = 0
    
    for step in range(max_steps_per_episode):
        # select an action
        # use our exploration exploitation trade off -> do we explore or exploit in this timestep ?
        exploration_rate_threshold = random.uniform(0,1)
        if(exploration_rate_threshold > exploration_rate):
            action = np.argmax(q_table[state, : ])
        else:
            action = env.action_space.sample()
            
        new_state, reward, done, info = env.step(action)
        
        # discretize the observed state
        new_state = discretize_state(new_state)
        
        # Update Q-Table Q(s,a) using the bellman update  
        q_table[state, action] = q_table[state, action] * (1 - learning_rate) + learning_rate * (reward + discount_rate * np.max(q_table[new_state, : ]))
        
        # update the state to the new state
        state = new_state
        # collect the reward
        rewards_current_episode += reward
        
        if (done == True):
            break
    
    # after we finish an episode, make sure to update the exploration rate
    # decay the exploration rate the longer the time goes on
    exploration_rate = min_exploration_rate + (max_exploration_rate - min_exploration_rate) * np.exp(-exploration_decay_rate*episode)
    # append our rewards for this episode for learning curve
    rewards_of_all_episodes.append(rewards_current_episode)

In [ ]:
# LEARNING STATISTICS 
# for each episode print the stats of the episode
rewards_per_thousand_episodes = np.split(np.array(rewards_of_all_episodes),num_episodes/1000)
count = 1000
print('*****INFO: average reward per thousand episodes: ***** \n')
for reward in rewards_per_thousand_episodes:
    print(count, ": ", str(sum(reward/1000)))
    count += 1000
    
# print our learned q-table
print("\n\n ***** Q-TABLE ***** \n")
print(q_table)

In [ ]:
# EVALUATION | TESTING | watching our agent play
for episode in range(num_test_episodes):
    state = env.reset()
    # discretize state
    state = discretize_state(state)
    done = False
    print("INFO:*****EPISODE ", episode+1, "\n\n\n")
    time.sleep(1)
    
    for step in range(max_steps_per_episode):
        clear_output(wait=True)
        env.render()
        time.sleep(0.1)
        
        action = np.argmax(q_table[state, :])
        new_state, reward, done, info = env.step(action)
        # discretize new state
        new_state = discretize_state(new_state)
        
        if done:
            clear_output(wait=True)
            env.render()
            if reward == 1: # check reward from environment for correct display
                print("INFO: ***** agent reached the goal. *****")
                time.sleep(3)
            else:
                print("INFO: ***** agent did not reach the goal.")
                time.sleep(3)
            clear_output(wait=True)
            break
        
        state = new_state

env.close()

***

#### Deep Q Networks

![alt_text](https://cdn.analyticsvidhya.com/wp-content/uploads/2019/04/Screenshot-2019-04-16-at-5.46.01-PM.png)

In [ ]:
import torch
from torch.autograd import Variable

In [ ]:
# get an instance of the frozen lake gym environment
env = gym.make("CartPole-v0")

In [ ]:
# get some information from the environment
action_space = env.action_space
print(action_space)
action_space_size = env.action_space.n
print(action_space_size)
state_space = env.observation_space 
print(state_space)
state_space_size = env.observation_space.shape[0]
print(state_space_size)

In [ ]:
# Training Parameters
num_episodes = 1000
max_steps_per_episode = 100
num_test_episodes = 1     

# q-learning | update parameters
discount_rate = 0.99

# exploration-exploitation trade off
exploration_rate = 1
max_exploration_rate = 1
min_exploration_rate = 0.01
exploration_decay_rate = 0.001

# q network parameters
learning_rate = 0.001
hidden_dim = 64

In [ ]:
class DeepQNetwork():
    ''' Deep Q Neural Network Class. '''
    def __init__(self, state_dim, action_dim, hidden_dim=64, lr=0.05):
            self.criterion = torch.nn.MSELoss()
            self.model = torch.nn.Sequential(
                            torch.nn.Linear(state_dim, hidden_dim),
                            torch.nn.LeakyReLU(),
                            torch.nn.Linear(hidden_dim, hidden_dim*2),
                            torch.nn.LeakyReLU(),
                            torch.nn.Linear(hidden_dim*2, action_dim)
                    )
            self.optimizer = torch.optim.Adam(self.model.parameters(), lr)

    def update(self, state, y):
            """Update the weights of the network given a training sample. """
            y_pred = self.model(torch.Tensor(state))
            loss = self.criterion(y_pred, Variable(torch.Tensor(y)))
            self.optimizer.zero_grad()
            loss.backward()
            self.optimizer.step()

    def predict(self, state):
            """ Compute Q values for all actions using the DeepQNetwork """
            with torch.no_grad():
                return self.model(torch.Tensor(state))

In [ ]:
# get our deep q network
deep_q_network = DeepQNetwork(state_space_size, action_space_size, hidden_dim, learning_rate)

In [ ]:
# Deep Q-LEARNING

# collect rewards somewhere to visualize our learning curve
rewards_of_all_episodes = []

# Training Loop
for episode in range(num_episodes):
    # reset/initialize the environment first
    state = env.reset()
    # set done back to false at the beginning of an episode
    done = False
    # reset our rewards collector | return for the beginning episode
    rewards_current_episode = 0
    
    for step in range(max_steps_per_episode):
        # select an action
        # use our exploration exploitation trade off -> do we explore or exploit in this timestep ?
        exploration_rate_threshold = random.uniform(0,1)
        if(exploration_rate_threshold > exploration_rate):
            q_values = deep_q_network.predict(state)
            action = torch.argmax(q_values).item()
        else:
            action = env.action_space.sample()
            
        new_state, reward, done, info = env.step(action)
        
        # Update our Q Network due to the reward we got
        q_values = deep_q_network.predict(state).tolist()
        # Update network weights using the last step only
        q_values_next = deep_q_network.predict(new_state)
        q_values[action] = reward + discount_rate * torch.max(q_values_next).item()
        deep_q_network.update(state, q_values)
        
        # update the state to the new state
        state = new_state
        # collect the reward
        rewards_current_episode += reward
        
        if (done == True):
            q_values[action] = reward
            # Update network weights
            deep_q_network.update(state, q_values)
            break
    
    # after we finish an episode, make sure to update the exploration rate
    # decay the exploration rate the longer the time goes on
    exploration_rate = min_exploration_rate + (max_exploration_rate - min_exploration_rate) * np.exp(-exploration_decay_rate*episode)
    # append our rewards for this episode for learning curve
    rewards_of_all_episodes.append(rewards_current_episode)

In [ ]:
# LEARNING STATISTICS 
# for each episode print the stats of the episode
rewards_per_hundred_episodes = np.split(np.array(rewards_of_all_episodes),num_episodes/100)
count = 100
print('*****INFO: average reward per thousand episodes: ***** \n')
for reward in rewards_per_hundred_episodes:
    print(count, ": ", str(sum(reward/100)))
    count += 100

In [ ]:
# EVALUATION | TESTING | watching our agent play
for episode in range(num_test_episodes):
    state = env.reset()
    done = False
    print("INFO:*****EPISODE ", episode+1, "\n\n\n")
    time.sleep(1)
    
    for step in range(max_steps_per_episode):
        clear_output(wait=True)
        env.render()
        time.sleep(0.1)
        
        q_values = deep_q_network.predict(state)
        action = torch.argmax(q_values).item()
        new_state, reward, done, info = env.step(action)
        
        if done:
            clear_output(wait=True)
            env.render()
            if reward == 1: # check reward from environment for correct display
                print("INFO: ***** agent reached the goal. *****")
                time.sleep(3)
            else:
                print("INFO: ***** agent did not reach the goal.")
                time.sleep(3)
            clear_output(wait=True)
            break
        
        state = new_state

env.close()

***
#### IN SHORT: 

**Beginners:** 
If possible discretize the state spaces/observation spaces of your environments(if dealing with continous environments) and use simple, straight forward implementations of the Q-Learning algorithm. 

**Advanced:**
Use a neural network to approximate the q-table of the continous environment (=learning a q-function) and implement the Q-Learning algorithm by being consistent to the environment and agent interfaces of the openai gym framework. 

***

# TODO's

1. Send your finished presentations (+ possibly annotated paper) by **Monday 12.00 AM/midnight** via email to henrik.voigt@uni-jena.de

2. Send your little HOMEWORK to henrik.voigt@uni-jena.de by using the naming convention: HOMEWORK_02_FIRSTNAME_LASTNAME.ipynb until **June 23th 12.00 AM/midnight**

***
